In [1]:
from itertools import product
from random import random, randrange
from math import pi, sqrt

In [2]:
def roll_dice():
    return randrange(1, 7)

def flip_coin(bias=0.5, heads=0, tails=1):
    return heads if random() < bias else tails

In [3]:
def bell_sum(A_H, A_T, B_H, B_T):
    return A_H*B_H + A_H*B_T + A_T*B_H - A_T*B_T

print('A_H  A_T  B_H  B_T    Bell-sum')
print('------------------------------')
values = [-1, +1]
for A_H, A_T, B_H, B_T in product(values, values, values, values):
    print(f' {A_H:2}   {A_T:2}   {B_H:2}   {B_T:2}    ->    {bell_sum(A_H, A_T, B_H, B_T):2}')

A_H  A_T  B_H  B_T    Bell-sum
------------------------------
 -1   -1   -1   -1    ->     2
 -1   -1   -1    1    ->     2
 -1   -1    1   -1    ->    -2
 -1   -1    1    1    ->    -2
 -1    1   -1   -1    ->     2
 -1    1   -1    1    ->    -2
 -1    1    1   -1    ->     2
 -1    1    1    1    ->    -2
  1   -1   -1   -1    ->    -2
  1   -1   -1    1    ->     2
  1   -1    1   -1    ->    -2
  1   -1    1    1    ->     2
  1    1   -1   -1    ->    -2
  1    1   -1    1    ->    -2
  1    1    1   -1    ->     2
  1    1    1    1    ->     2


In [4]:
# version 1 of the bell_experiment() function, where we compute just one sum
# this works assuming N is large enough so the 4 cases HH, HT, TH, and TT
# all occur roughly N/4 times
#
def bell_experiment(N):
    total = 0
    for _ in range(N):
        # Generate a composite object and split it
        composite = generate_composite()
        left, right = split(composite)

        # Alice's coin flip determines which measurement device she uses
        A_which = flip_coin()
        measure_A = measure_A_H if A_which == 0 else measure_A_T
        A = measure_A(left)

        # Bob's coin flip determines which measurement device he uses
        B_which = flip_coin()
        measure_B = measure_B_H if B_which == 0 else measure_B_T
        B = measure_B(right)

        # Compute the contribution to the Bell statistic
        multiplier = -1 if A_which == 1 and B_which == 1 else 1
        total += multiplier * A * B

    # Calculate the Bell statistic and print the result
    bell_statistic = 4 * float(total) / N
    print(f'Bell Statistic: {bell_statistic:.3}')

In [5]:
# version 2 of the bell_experiment() function, where we maintain four separate sums
#
def bell_experiment(N):
    cnts = [[0, 0], [0, 0]]
    sums = [[0, 0], [0, 0]]
    for _ in range(N):
        # we create some composite system
        composite = generate_composite()
        # .. and split it in two parts, which we'll label 'left' and 'right'
        #    (left and right have no significance, we could equally well call it 1 and 2 or u and w)
        left, right = split(composite)
        # we send the left part to Alice, right part to Bob
        # ..
        # Alice flips a coin to decide whether she uses the H or T measurement apparatus
        A_which = flip_coin()
        measure_A = measure_A_H if A_which == 0 else measure_A_T
        # Alice measures her part of the composite system
        A = measure_A(left)
        # ..
        # .. meanwhile
        # Bob flips a coin to decide whether he uses the H or T measurement apparatus
        B_which = flip_coin()
        measure_B = measure_B_H if B_which == 0 else measure_B_T
        B = measure_B(right)
        # Alice sends back her measurement result A, as well as which measurement device she used, A_which
        #   Bob sends back his measurement result B, as well as which measurement device she used, B_which
        AB = A*B
        # record results in appropriate bucket
        cnts[A_which][B_which] += 1
        sums[A_which][B_which] += AB
    # ...
    # experiment is over, compute Bell statistic
    # print(sums[0][0], sums[1][0], sums[0][1], sums[1][1])
    # print(cnts[0][0], cnts[1][0], cnts[0][1], cnts[1][1])
    bell_statistic = (sums[0][0]/cnts[0][0] +
                      sums[1][0]/cnts[1][0] +
                      sums[0][1]/cnts[0][1] -
                      sums[1][1]/cnts[1][1])
    print(f'Bell Statistic: {bell_statistic:.3}')

In [6]:
# the measurement functions are constant with signature (1, 1, 1, 1)

def generate_composite():
    return None, None

def split(composite):
    left = composite[0]
    right = composite[1]
    return left, right
    
def measure_A_H(_):
    return 1

def measure_A_T(_):
    return 1

def measure_B_H(_):
    return 1
    
def measure_B_T(_):
    return 1

bell_experiment(N = 1000*1000)

Bell Statistic: 2.0


In [7]:
# the measurement functions are constant with signature (1, -1, 1, -1)

def measure_A_H(_):
    return 1

def measure_A_T(_):
    return -1

def measure_B_H(_):
    return 1
    
def measure_B_T(_):
    return -1

bell_experiment(N = 1000*1000)

Bell Statistic: -2.0


In [8]:
def generate_composite():
    d = roll_dice()
    composite = (d, d)
    return composite

def measure_A_H(q):
    result = 1 if q <= 3 else -1
    return result

def measure_A_T(q):
    result = 1 if q % 2 == 0 else -1
    return result

measure_B_H = measure_A_H
measure_B_T = measure_A_T

bell_experiment(N = 1000*1000)

Bell Statistic: -0.662


In [9]:
def generate_composite():
    d = roll_dice()
    composite = (d, d)
    return composite

def measure_A_H(q):
    result = 1 if q <= 3 else -1
    return result

def measure_A_T(q):
    result = 1 if q % 2 == 0 else -1
    return result

measure_B_H = measure_A_T
measure_B_T = measure_A_H

bell_experiment(N = 1000*1000)

Bell Statistic: 2.0


In [10]:
def generate_composite():
    d = roll_dice()
    composite = (d, d if d <= 4 else flip_coin(heads=5, tails=6))
    return composite

def measure_A_H(q):
    result = 1 if q <= 4 else -1
    return result

def measure_A_T(q):
    result = 1 if q % 2 == 0 else -1
    return result

measure_B_H = measure_A_H
measure_B_T = measure_A_T

bell_experiment(N = 1000*1000)

Bell Statistic: 0.336


In [11]:
measure_B_H = measure_A_T
measure_B_T = measure_A_H

bell_experiment(N = 1000*1000)

Bell Statistic: 1.66


In [12]:
# introducing randomness in Bob's measurement function
# this no longer simulates a DETERMINISTIC local hidden variable theory
# but it still cannot break the Bell inequality

def generate_composite():
    d = roll_dice()
    composite = (d, d if d <= 4 else flip_coin(heads=5, tails=6))
    return composite

def measure_A_H(q):
    result = 1 if q <= 4 else -1
    return result

def measure_A_T(q):
    result = 1 if q % 2 == 0 else -1
    return result

measure_B_H = measure_A_T

def measure_B_T(q):
    result = flip_coin(heads=1, tails=-1)
    return result

bell_experiment(N = 1000*1000)

Bell Statistic: 0.665


In [13]:
def generate_composite():
    d = roll_dice()
    composite = (d, d if d <= 4 else flip_coin(heads=5, tails=6))
    return composite

def split(composite):
    left  = {'value': composite[0]}
    right = {'value': composite[1]}
    # we enable both left and right parts to peek at each other
    # in physics, this is called action-at-a-distance
    left['other'] = right
    right['other'] = left
    return left, right

def measure_A_H(q):
    # the particle remembers which measurement was performed and what the result was
    # in itself, this is not cheating
    q['measure'] = 'H'
    q['result'] = 1 if q['value'] <= 4 else -1
    return q['result']

def measure_A_T(q):
    q['measure'] = 'T'
    q['result'] = 1 if q['value'] % 2 == 0 else -1
    return q['result']

def measure_B_H(q):
    # always return what Alice measured
    q['result'] = q['other']['result']
    return q['result']

def measure_B_T(q):
    # condition on which measurement Alice performced and use her result
    q['result'] = q['other']['result'] if q['other']['measure'] == 'H' else -1*q['other']['result']
    return q['result']

bell_experiment(N = 1000*1000)

Bell Statistic: 4.0


In [14]:
def generate_composite():
    # we generate pairs of particles with anti-aligned spins in the singlet state
    # the line of code below doesn't matter, it's just for documentation
    return {'state': '(|01>-|10>)/√2'}
    # |0> is the eigenstate of the Pauli operator σ_z corresponding to eigenvalue (=measurement outcome) +1
    # |1> is the eigenstate of the Pauli operator σ_z corresponding to eigenvalue (=measurement outcome) -1
    # |+> is the eigenstate of the Pauli operator σ_x corresponding to eigenvalue (=measurement outcome) +1
    # |-> is the eigenstate of the Pauli operator σ_x corresponding to eigenvalue (=measurement outcome) -1
    # the above state can be rewritten in terms of |+> and |->, like:
    # (|01>-|10>)/√2  =  (|-+>-|+->)/√2
    
def split(composite):
    # in quantum mechanics, there is no sense to talk about a left and right
    # value (before measurement) of a singlet composite system..
    left  = {'value': None}
    right = {'value': None}
    # we enable both left and right parts to peek at each other
    # in physics, this is called action-at-a-distance
    left['other'] = right
    right['other'] = left
    return left, right

def measure_first_qubit(q, orientation):
    q['measure'] = orientation
    # Alice measures using σ_z or σ_x
    # in both cases, the amplitudes of the 2 possible states are equal = 1/√2,
    # so the probability of both measurement outcomes is 1/2
    # but the post-measurement state is not the same in the 4 cases (2 measurements, each 2 outcomes ±1
    # if Alice uses σ_z and measures +1 on the first qubit, the post measurement state of the composite system is |01>
    # if Alice uses σ_z and measures -1 on the first qubit, the post measurement state of the composite system is |10>
    # if Alice uses σ_x and measures +1 on the first qubit, the post measurement state of the composite system is |+->
    # if Alice uses σ_x and measures -1 on the first qubit, the post measurement state of the composite system is |-+>
    q['result'] = flip_coin(heads=1, tails=-1)
    return q['result']

measure_A_H = lambda q: measure_first_qubit(q, 'H') # Alice uses the Pauli operator σ_z to measure the first qubit
measure_A_T = lambda q: measure_first_qubit(q, 'T') # Alice uses the Pauli operator σ_x to measure the first qubit

p = 1/(4-2*sqrt(2))
def measure_second_qubit(q, orientation):
    # Bob's H corresponds to using the operator -(σ_x+σ_z)/√2 to measure the second qubit
    # Bob's T corresponds to using the operator  (σ_x-σ_z)/√2 to measure the second qubit
    # the math here is straightforward quantum theory:
    # let's look at an example:
    # suppose Alice flipped a coin and used σ_z (which we labeled Alice's H) and measured +1 on the first qubit, leaving the composite system in the state |01>
    # suppose that now Bob flips a coin and uses -(σ_x+σ_z)/√2 (which we labeled Bob's H):
    # the challenge now is to express the composite state |01> in terms of the eigenvectors of Bob's measurement operator -(σ_x+σ_z)/√2
    # suppose the 2 eigenvectors are U and V with eigenvalues +1 and -1, and we can write |01> = u*|U> + v*|V>, where u and v are the complex amplitudes
    # this can be done on a piece of paper, to find u, v, U, V, and then the probabilities of outcome +1 is p=u^2 and -1 is 1-p=v^2=1-u^2
    # .. where p is the number defined above p = 1/(4-2*sqrt(2)), because I did the math on paper
    # there are 16 combinations of Alice and Bob measurements and outcomes (2 people, each 2 measurements, each measurement 2 outcomes)
    # .. which can be neatly written into a table, and the probability is always p or 1-p
    # see:
    #     https://docs.google.com/spreadsheets/d/1MfsGAqPoN1gK5iA3veipgoaSQ_mhqOL4bLhfYCqy_Sk
    # 
    # the code below encodes this table
    q['measure'] = orientation
    sign = 1 if orientation == 'H' or q['other']['measure'] != orientation else -1
    if q['other']['result'] == 1:
        q['result'] = sign * flip_coin(p, 1, -1)
    else:
        q['result'] = sign * flip_coin(p, -1, 1)
    return q['result']

measure_B_H = lambda q: measure_second_qubit(q, 'H')
measure_B_T = lambda q: measure_second_qubit(q, 'T')

bell_experiment(N = 1000*1000)

Bell Statistic: 2.83
